In [2]:
from pyspark.sql.session import SparkSession
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
from pyspark.sql.types import *
from pyspark.sql.functions import concat,concat_ws
from pyspark.sql.functions import *
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
# nltk.download('punkt')
# nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
# nltk.download('wordnet')
import pandas as pd
import sklearn as sk
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
product_path='D:/PROJECTS/FASHIONPASS/Database/product.csv'
tagcloud_path='D:/PROJECTS/FASHIONPASS/Database/tag_cloud.csv'
collection_category_tag_path='D:/PROJECTS/FASHIONPASS/Database/collection_category_tag.csv'
product_tag_path='D:/PROJECTS/FASHIONPASS/Database/product_tag.csv'
features_path='D:/PROJECTS/FASHIONPASS/Database/spars_dense_features.csv'
collection_category_path='D:/PROJECTS/FASHIONPASS/Database/collection_category.csv'
clothing_collection_path='D:/PROJECTS/FASHIONPASS/Database/cloting_collection_category.csv'
product_cloth_collection_path='D:/PROJECTS/FASHIONPASS/Database/product_cloth_cloection_tag.csv'
sku_path='D:/PROJECTS/FASHIONPASS/Database/sku.csv'

In [4]:
spark = SparkSession.builder \
    .appName("PySpark MySQL Example") \
    .config("spark.driver.extraClassPath", "C:/spark/spark-3.5.0-bin-hadoop3/jars/mysql-connector-java-5.1.46.jar") \
    .getOrCreate()



In [6]:
Product_df=spark.read.format('csv').option('header',True) \
.load(product_path)

Tagcloud_df=spark.read.format('csv').option('header',True) \
.load(tagcloud_path)


Collection_Category_tag_df=spark.read.format('csv').option('header',True) \
.load(collection_category_tag_path)

Product_tag_df=spark.read.format('csv').option('header',True) \
.load(product_tag_path)


features_df=spark.read.format('csv').option('header',True) \
.load(features_path)


Collection_Category_df=spark.read.format('csv').option('header',True)\
.load(collection_category_path)

clothing_collection_df=spark.read.format('csv').option('header',True)\
.load(clothing_collection_path)


sku_df=spark.read.format('csv').option('header',True)\
.load(sku_path)
# product_cloth_collection_tag_df=spark.read.format('csv').option('header',True)\
# .load(product_cloth_collection_path)

In [29]:
sku_df=sku_df.select(['product_id','option2'])
product_color=sku_df.dropDuplicates()


+----------+-------+
|product_id|option2|
+----------+-------+
|      1192|  BLACK|
+----------+-------+



In [32]:
product_color_string=[ str(string[0]).lower() for string in product_color.select('option2').collect()]
product_id=[ id[0] for id in  product_color.select('product_id').collect()]

In [34]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('bert-base-nli-mean-tokens')

# Encode sentences
embeddings = model.encode(product_color_string, convert_to_tensor=True)

In [35]:
cos_sim = util.cos_sim(embeddings, embeddings)

cos_sim

tensor([[1.0000, 0.5021, 0.5155,  ..., 0.5155, 0.3785, 0.5574],
        [0.5021, 1.0000, 0.7483,  ..., 0.7483, 0.4775, 0.6535],
        [0.5155, 0.7483, 1.0000,  ..., 1.0000, 0.4646, 0.6071],
        ...,
        [0.5155, 0.7483, 1.0000,  ..., 1.0000, 0.4646, 0.6071],
        [0.3785, 0.4775, 0.4646,  ..., 0.4646, 1.0000, 0.6270],
        [0.5574, 0.6535, 0.6071,  ..., 0.6071, 0.6270, 1.0000]])

In [36]:
all_product_combination = {'cosin_score_color':[] , 'Product_A':[] , 'Product_B':[]}
for i in range(len(cos_sim)-1):
    for j in range(i+1, len(cos_sim)):
        all_product_combination['cosin_score_color'].append(float(cos_sim[i][j]))
        all_product_combination['Product_A'].append(product_id[i])
        all_product_combination['Product_B'].append(product_id[j])

In [37]:
recomendation_df=pd.DataFrame(all_product_combination)

In [47]:
recomendation_df.to_csv('D:/PROJECTS/FASHIONPASS/cosin_similarity_registory/cosine_similarity_product_color_1_9_2023.csv')

In [49]:
product_title='SAILOR PANT'
selected_product_id=Product_df.filter(Product_df['product_title']==product_title).select('product_id').collect()[0][0]
recomendation_df_product=recomendation_df[recomendation_df['Product_A']==selected_product_id]
link=[]
for index,row in recomendation_df_product.iterrows():
    if row['cosin_score_color']>=.90:
        id=row['Product_B']
        # link.append(int(id))
        # product_thumbnail=Product_df.filter(Product_df.product_id==id).select('product_thumbnail').collect()
        try:
            # print('https://fashionpass.s3.us-west-1.amazonaws.com/products/'+(product_thumbnail[0][0]))
             link.append(id)
        except:
            print('valid string')    
print(','.join(link))

In [51]:
recomendation_df_product

,cosin_score_color,Product_A,Product_B
12774300,0.589396,7641,7879
12774301,0.397006,7641,8036
12774302,0.601015,7641,8137
12774303,0.537408,7641,8460
12774304,0.564079,7641,8496
...,...,...,...
12781386,0.598916,7641,1243
12781387,0.592194,7641,4333
12781388,0.537408,7641,5596
12781389,0.522239,7641,887
